In [1975]:
import numpy as np
import pandas as pd
import re
# from tqdm import tqdm
# from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [1976]:
c_fraud_df = pd.read_csv("corrected_names.csv")
c_fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17590 entries, 0 to 17589
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Caller Number            17590 non-null  object
 1   Call Time                17590 non-null  object
 2   Call Duration(in s)      17590 non-null  int64 
 3   Call Frequency Per Day   17590 non-null  int64 
 4   Call Frequency Per Week  17590 non-null  int64 
 5   Call Type                17590 non-null  object
 6   Conversation             17590 non-null  object
dtypes: int64(3), object(4)
memory usage: 962.1+ KB


In [1977]:
c_fraud_df['Call Time'] = pd.to_datetime(c_fraud_df['Call Time'], format='%H:%M:%S')
c_fraud_df.dtypes

Caller Number                      object
Call Time                  datetime64[ns]
Call Duration(in s)                 int64
Call Frequency Per Day              int64
Call Frequency Per Week             int64
Call Type                          object
Conversation                       object
dtype: object

In [1978]:
c_fraud_df.head()

,Caller Number,Call Time,Call Duration(in s),Call Frequency Per Day,Call Frequency Per Week,Call Type,Conversation
0,+1-917-555-3286,1900-01-01 08:50:33,62,3,25,Normal,"Hope. Since ago travelled, he has forgotten hi..."
1,+1-205-795-8028,1900-01-01 17:05:53,78,3,31,Normal,Horrible you eat mass eat until u forgot about...
2,+1-702-412-9422,1900-01-01 16:29:51,55,4,7,Normal,Great! I have to run now so tell!
3,+1-915-426-2998,1900-01-01 17:36:33,48,9,14,Normal,"Sorry, I'll call later"
4,+1-915-543-9437,1900-01-01 20:19:36,262,3,16,Normal,"May wants to work out first, how's 4 sound?"


## Splitting the data

In [1979]:
from sklearn.model_selection import train_test_split

def split_data_stratified(X, y, test_size=0.2, eval_size=0.1, random_state=42):
    
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=test_size + eval_size, random_state=random_state, stratify=y)
    
    X_eval, X_test, y_eval, y_test = train_test_split(
        X_temp, y_temp, test_size=test_size / (test_size + eval_size), random_state=random_state, stratify=y_temp)
    
    del X_temp,y_temp
    
    return X_train, X_eval, X_test, y_train, y_eval, y_test


In [1980]:
#Dividing columns to features and labels
y = c_fraud_df['Call Type']
X = c_fraud_df.drop(columns='Call Type', axis=1)

In [1981]:
X_train, X_eval, X_test, y_train, y_eval, y_test = split_data_stratified(X,y)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [1982]:
X_train.head()

,Caller Number,Call Time,Call Duration(in s),Call Frequency Per Day,Call Frequency Per Week,Conversation
0,+1-917-436-9480,1900-01-01 12:02:10,1325,5,10,whiskey Brandy Sum In Beer Vodka Scotch Shampa...
1,+1-718-745-1135,1900-01-01 12:05:16,1572,12,30,He detected a failed login attempt on your ba...
2,+1-213-329-9734,1900-01-01 22:06:51,445,1,26,"Up to you, you want come then come for... But ..."
3,+1-628-329-7445,1900-01-01 21:20:38,243,5,17,My Anna is breaking me out. The's looked thou ...
4,+1-417-861-6252,1900-01-01 09:32:34,373,4,16,Key are you angry with me. Reply me dr.


In [1983]:
X_train.dtypes

Caller Number                      object
Call Time                  datetime64[ns]
Call Duration(in s)                 int64
Call Frequency Per Day              int64
Call Frequency Per Week             int64
Conversation                       object
dtype: object

## Oversampling Data to handle data imbalance

In [1984]:
a = y_train.eq('Spam').sum()
b = y_train.eq('Scam').sum()
print(a)
print(b)

1206
325


In [1985]:
rsa = RandomOverSampler( sampling_strategy={'Spam': 1757,'Scam':683},random_state=42)

X_resampled, y_resampled = rsa.fit_resample(X_train, y_train)

In [1986]:
X_resampled.head()

,Caller Number,Call Time,Call Duration(in s),Call Frequency Per Day,Call Frequency Per Week,Conversation
0,+1-917-436-9480,1900-01-01 12:02:10,1325,5,10,whiskey Brandy Sum In Beer Vodka Scotch Shampa...
1,+1-718-745-1135,1900-01-01 12:05:16,1572,12,30,He detected a failed login attempt on your ba...
2,+1-213-329-9734,1900-01-01 22:06:51,445,1,26,"Up to you, you want come then come for... But ..."
3,+1-628-329-7445,1900-01-01 21:20:38,243,5,17,My Anna is breaking me out. The's looked thou ...
4,+1-417-861-6252,1900-01-01 09:32:34,373,4,16,Key are you angry with me. Reply me dr.


In [1987]:
y_resampled.head()

0    Normal
1      Scam
2    Normal
3    Normal
4    Normal
Name: Call Type, dtype: object

In [1988]:
X_resampled.dtypes

Caller Number                      object
Call Time                  datetime64[ns]
Call Duration(in s)                 int64
Call Frequency Per Day              int64
Call Frequency Per Week             int64
Conversation                       object
dtype: object

## Pipelining


Function to create two new columns as decided in preprocessing steps i.e Area Code, Hour and Binned Hour values


In [1989]:
class AddAreaCodeDropNumber(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        
        X['Area Code'] = X['Caller Number'].str[3:6]

        def clean_phone_number(phone_number):
            cleaned_number = re.sub(r'[^0-9]', '', phone_number)
            return cleaned_number[-7:]

        X['PhNoLastDig'] = X['Caller Number'].apply(clean_phone_number)

        X['Area Code'] = X['Area Code'].astype(int)
        X['PhNoLastDig'] = X['PhNoLastDig'].astype(int)
        
        X.drop(columns=['Caller Number'], inplace=True)
        
        
        return X


In [1990]:
class AddHourDropTime(BaseEstimator, TransformerMixin):
    def fit(self,X,y = None):
        return self
    
    def transform(self, X):

        X['Hour'] = X['Call Time'].dt.hour.astype(int)

        X.drop(columns='Call Time', inplace=True)

        return X

In [1991]:
class TimeOfDayTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        if 'Hour' not in X.columns:
            raise ValueError("The input DataFrame must contain an 'Hour' column.")

        X['TimeOfDay'] = pd.cut(X['Hour'], bins=[0, 7, 9, 12, 15, 17, 20, 23], labels=['0-7', '7-9', '9-12', '12-15','15-17','17-20','20-23'], right=False)

        return X

In [1992]:
class StemmingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.port_stem = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, (list, pd.Series)):
            raise ValueError("Input must be a pandas Series or a list of strings.")

        return X.apply(self._stem_content)
    
    def _stem_content(self, content):
        stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
        stemmed_content = stemmed_content.lower()
        stemmed_content = stemmed_content.split()
        stemmed_content = [
            self.port_stem.stem(word) 
            for word in stemmed_content 
            if word not in self.stop_words
        ]
        return ' '.join(stemmed_content)

Function to split dataset into numerical and categorical datasets to build separate pipelines

Pipeline Construction

In [1993]:
AddAreaCodeCol = Pipeline(steps=[
    ('Add_AreaCode_Column', AddAreaCodeDropNumber()),
    # ('scaler', StandardScaler())
])

In [1994]:
AddHourCol = Pipeline(steps=[
    ('Add_Hour_Column', AddHourDropTime())
])

In [1995]:
AddBinnedTimeCol = Pipeline(steps=[
    ('Add_time_binned', TimeOfDayTransformer())
])

In [1996]:
time_pipeline = Pipeline(steps=[
    ('hour', AddHourCol),               # Extract 'Hour' from 'Call Time'
    ('binned_time', AddBinnedTimeCol)  # Bin 'Hour' into categories
    # ('scaler', StandardScaler())
])

In [1997]:
numerical_transformer = Pipeline(steps=[
    ('Imputation_median',SimpleImputer(missing_values=np.nan,strategy='median')),
    # ('scaler', StandardScaler())
])

In [1998]:
categorical_transformer = Pipeline(steps=[
    ('Text_imputation', SimpleImputer(fill_value='', strategy='constant')),
    ('OneHot',OneHotEncoder())
])

In [1999]:
text_transformer = Pipeline(steps=[
    # ('stemming', StemmingTransformer())
    ('vectorizer', TfidfVectorizer())  
])

Main Pipeline

In [2000]:
add_data = Pipeline(steps=[
    ('AreaCode',AddAreaCodeCol),
    ('Time', time_pipeline) #Creates three new columns AreaCode, LastPhNo and TimeOfDay
])

In [2001]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['Call Duration(in s)','Call Frequency Per Day','Call Frequency Per Week','Area Code','PhNoLastDig','Hour']),
        ('cat', categorical_transformer, ['TimeOfDay']),
        # ('text', text_transformer, ['Conversation'])
    ]
)

In [2002]:
data_pipeline = Pipeline(steps=[
    ('appendandremovecol', add_data),
    ('preprocessor', preprocessor)    
])


In [2003]:
X_preprocessed = data_pipeline.fit_transform(X_resampled)
X_preprocessed = pd.DataFrame(X_preprocessed)

In [2004]:
X_preprocessed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1325.0,5.0,10.0,917.0,4369480.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1572.0,12.0,30.0,718.0,7451135.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,445.0,1.0,26.0,213.0,3299734.0,22.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,243.0,5.0,17.0,628.0,3297445.0,21.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,373.0,4.0,16.0,417.0,8616252.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13216,54.0,7.0,72.0,572.0,2839686.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13217,25.0,13.0,38.0,267.0,6125753.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13218,27.0,5.0,29.0,415.0,8141250.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13219,21.0,4.0,30.0,323.0,2478497.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [2005]:
# X_transformed.dtypes

In [2006]:
set_config(display="diagram")
data_pipeline

Pipeline(steps=[('appendandremovecol',
                 Pipeline(steps=[('AreaCode',
                                  Pipeline(steps=[('Add_AreaCode_Column',
                                                   AddAreaCodeDropNumber())])),
                                 ('Time',
                                  Pipeline(steps=[('hour',
                                                   Pipeline(steps=[('Add_Hour_Column',
                                                                    AddHourDropTime())])),
                                                  ('binned_time',
                                                   Pipeline(steps=[('Add_time_binned',
                                                                    TimeOfDayTransformer())]))]))])),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Imputation_median',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Call Duration(in s)',
                                                   'Call Frequency Per Day',
                                                   'Call Frequency Per Week',
                                                   'Area Code', 'PhNoLastDig',
                                                   'Hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('Text_imputation',
                                                                   SimpleImputer(fill_value='',
                                                                                 strategy='constant')),
                                                                  ('OneHot',
                                                                   OneHotEncoder())]),
                                                  ['TimeOfDay'])]))])